In [3]:
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT, KEYWORD
from whoosh.qparser import QueryParser
import os
import difflib



Query: 사랑니 발치했어요.
Code: K02.1, Name: 매복 사랑니, Category: 수술, Detail: 매복 사랑니 발치
===
Query: 충치 치료는 어떻게 하나요?
Code: K01.0, Name: 유치의 충치, Category: 진료, Detail: 충치 치료
Code: K04.0, Name: 전치부 충치, Category: 진료, Detail: 전치부 충치 치료
===
Query: 턱관절이 아프다.
No results found.
===
Query: 사악
No results found.
===
Query: 구강 염증 치료가 필요해요.
Code: K06.0, Name: 구강내 염증, Category: 치료, Detail: 구강 염증 치료
Code: K11.0, Name: 치근단 염증, Category: 수술, Detail: 치근단 염증 치료
Code: K00.0, Name: 선천성 구강병, Category: 진료, Detail: 구강 검진
Code: K07.0, Name: 구강의 기타 질환, Category: 치료, Detail: 기타 치료
Code: K09.0, Name: 구강 궤양, Category: 진료, Detail: 구강 궤양 치료
===
Query: 발치가 필요할까요?
No results found.
===
Query: 치료 방법은 뭐가 있을까?
No results found.
===
Query: 매복 사랑니가 너무 아파요.
Code: K02.1, Name: 매복 사랑니, Category: 수술, Detail: 매복 사랑니 발치
===
Query: 구강 검진 예약하고 싶어요.
Code: K00.0, Name: 선천성 구강병, Category: 진료, Detail: 구강 검진
Code: K06.0, Name: 구강내 염증, Category: 치료, Detail: 구강 염증 치료
Code: K07.0, Name: 구강의 기타 질환, Category: 치료, Detail: 기타 치료
Code: K09.0, Name: 구강 궤양, C

In [ ]:

# 인덱스 디렉토리 생성
if not os.path.exists("indexdir"):
    os.mkdir("indexdir")

# 스키마 정의
schema = Schema(
    incomplete_disease_code=KEYWORD(stored=True),
    incomplete_disease_name=TEXT(stored=True),
    claim_category=TEXT(stored=True),
    claim_detail=TEXT(stored=True)
)

In [ ]:
# 인덱스 생성
ix = create_in("indexdir", schema)

# 데이터 색인
writer = ix.writer()
data = {
    "incomplete_disease_code": [
        "K00.0", "K00.1", "K01.0", "K02.0", "K02.1", 
        "K03.0", "K03.1", "K04.0", "K05.0", "K05.1",
        "K06.0", "K06.1", "K07.0", "K08.0", "K09.0",
        "K10.0", "K11.0", "K12.0", "K13.0", "K14.0"
    ],
    "incomplete_disease_name": [
        "선천성 구강병", "선천성 악질환", "유치의 충치", "상악의 감염", "매복 사랑니",
        "치주염", "치은염", "전치부 충치", "상악 치조골의 질병", "하악의 감염",
        "구강내 염증", "입안의 종양", "구강의 기타 질환", "턱관절 장애", "구강 궤양",
        "급성 치통", "치근단 염증", "악골의 낭종", "하악 골절", "상악 골절"
    ],
    "claim_category": [
        "진료", "수술", "진료", "치료", "수술",
        "치료", "치료", "진료", "진료", "수술",
        "치료", "치료", "치료", "수술", "진료",
        "진료", "수술", "치료", "진료", "진료"
    ],
    "claim_detail": [
        "구강 검진", "사랑니 발치", "충치 치료", "치아 발치", "매복 사랑니 발치",
        "치주 치료", "치은 치료", "전치부 충치 치료", "치조골 치료", "하악 감염 치료",
        "구강 염증 치료", "종양 제거", "기타 치료", "턱관절 치료", "구강 궤양 치료",
        "급성 치통 치료", "치근단 염증 치료", "악골 낭종 제거", "하악 골절 치료", "상악 골절 치료"
    ]
}

# 색인 데이터 추가
for code, name, category, detail in zip(
    data["incomplete_disease_code"],
    data["incomplete_disease_name"],
    data["claim_category"],
    data["claim_detail"]
):
    writer.add_document(
        incomplete_disease_code=code,
        incomplete_disease_name=name,
        claim_category=category,
        claim_detail=detail
    )

writer.commit()

In [ ]:
# 오타 교정 기능
def correct_spelling(query):
    all_diseases = data["incomplete_disease_name"]
    matches = difflib.get_close_matches(query, all_diseases, n=1, cutoff=0.6)
    return matches[0] if matches else query

# 검색 함수
def search_claims(natural_query):
    # 오타 교정
    corrected_query = correct_spelling(natural_query)

    # 여러 키워드 처리
    keywords = corrected_query.split()
    queries = [f"{keyword}*" for keyword in keywords]  # 부분 일치 검색
    
    with ix.searcher() as searcher:
        # 각 키워드를 OR 조건으로 결합
        query_string = " OR ".join(queries)
        query = QueryParser("incomplete_disease_name", ix.schema).parse(query_string)
        results = searcher.search(query)

        # 결과 가중치 조정 (상위 5개 결과만 반환)
        ranked_results = sorted(results, key=lambda r: r.score, reverse=True)[:5]
        return [
            {
                "code": result['incomplete_disease_code'],
                "name": result['incomplete_disease_name'],
                "category": result['claim_category'],
                "detail": result['claim_detail']
            }
            for result in ranked_results
        ]

# 테스트 쿼리
queries = [
    "사랑니 발치했어요.",
    "충치 치료는 어떻게 하나요?", 
    "턱관절이 아프다.",
    "사악",
    "구강 염증 치료가 필요해요.", 
    "발치가 필요할까요?", 
    "치료 방법은 뭐가 있을까?", 
    "매복 사랑니가 너무 아파요.", 
    "구강 검진 예약하고 싶어요.", 
    "종양 제거 수술",
    "치주염 치료는 어디서 받지?",
    "K01.0",
    "치아",
    "치주 치료",
    "사랑니",  # 오타 예시
    "cldk"  # 잘못된 입력 예시
]

# 각 쿼리로 검색 수행
for query in queries:
    results = search_claims(query)
    print(f"Query: {query}")
    if results:
        for result in results:
            print(f"Code: {result['code']}, Name: {result['name']}, Category: {result['category']}, Detail: {result['detail']}")
    else:
        print("No results found.")
    print("===")